In [25]:
from rl_cardiac.tcn_model import TCN_config
from rl_cardiac.cardiac_model import CardiacModel_Env
rat_type = 'healthy_stable'
tcn_model = TCN_config(rat_type)
env = CardiacModel_Env(tcn_model, rat_type)
# noise level is set to 0 by default, should be changed to see if your agent can handle noise once it works well without noise
# env = CardiacModel_Env(tcn_model, rat_type, noise_level) 

script_dir /home/rl_course/rl_cardiac
[tcn_model/healthy_stable]
tcn_model healthy_stable
Load model from /home/rl_course/rl_cardiac/Healthy_500samples/TCN_1
[cardiac_model/healthy_stable]


In [26]:
from rl_cardiac.cardiac_model import CardiacModel_Env
# class testenv(CardiacModel_Env):
#     def reset(self,seed=None):
#         if seed is not None:
#             self.seed = seed
#         return super().reset(), []
rat_type = 'healthy_stable'
tcn_model = TCN_config(rat_type)
##env = CardiacModel_Env(tcn_model, rat_type)

script_dir /home/rl_course/rl_cardiac
[tcn_model/healthy_stable]
tcn_model healthy_stable
Load model from /home/rl_course/rl_cardiac/Healthy_500samples/TCN_1


In [27]:
env.reset()
for i in range(1000):
    action = env.action_space.sample()
    state, reward, _, done, _ = env.step(action)
    if done:
        break

seed:  None
1/1 [==============================] - 0s 13ms/step


In [14]:
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.results_plotter import load_results, ts2xy
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """

    def __init__(self, check_freq: int, log_dir: str, verbose=1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, "best_model")
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

            # Retrieve training reward
            x, y = ts2xy(load_results(self.log_dir), "timesteps")
            if len(x) > 0:
                # Mean training reward over the last 100 episodes
                mean_reward = np.mean(y[-100:])
                if self.verbose > 0:
                    print(f"Num timesteps: {self.num_timesteps}")
                    print(
                        f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}"
                    )

                # New best model, you could save the agent here
                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    # Example for saving best model
                    if self.verbose > 0:
                        print(f"Saving new best model to {self.save_path}.zip")
                    self.model.save(self.save_path)

        return True

In [ ]:
import os
import numpy as np
import torch
import torch.optim as optim
from TD3 import TD3
from utils import ReplayBuffer
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.results_plotter import load_results, ts2xy
from rl_cardiac.tcn_model import TCN_config
from rl_cardiac.cardiac_model import CardiacModel_Env


class TrainingProgressCallback(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)
        self.best_mean_reward = -np.inf
        self.episode_rewards = []

    def _on_step(self) -> bool:
        # This method is called after each step
        return True

    def _on_rollout_end(self):
        # This method is called after each episode
        if len(self.locals['episode_rewards']) > 0:
            last_episode_reward = self.locals['episode_rewards'][-1]
            self.episode_rewards.append(last_episode_reward)
            
            # Calculate mean reward over last 10 episodes
            if len(self.episode_rewards) >= 10:
                mean_reward = np.mean(self.episode_rewards[-10:])
                
                # Print progress and track best mean reward
                print(f"Last 10 episodes mean reward: {mean_reward:.2f}")
                
                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    print(f"New best mean reward: {self.best_mean_reward:.2f}")

def train_td3(env, num_episodes=1000, max_timesteps=1000):
    # Hyperparameters
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    max_action = float(env.action_space.high[0])
    print('state_dim: ', state_dim)
    print('action_dim:', action_dim)
    print('max_action:', max_action)
    
    # Initialize TD3 agent
    lr = 0.001
    agent = TD3(lr, state_dim, action_dim, max_action)
    
    # Replay buffer
    replay_buffer = ReplayBuffer()
    
    # Training loop
    for episode in range(num_episodes):
        # Use a more flexible reset approach
        reset_result = env.reset()
        
        # Handle different possible return types
        if isinstance(reset_result, tuple):
            # If multiple return values, take the first one
            state = reset_result[0]
        else:
            # If single value, use it directly
            state = reset_result
        
        episode_reward = 0
        
        for t in range(max_timesteps):
            # Select action
            action = agent.select_action(state)
            
            # Take step in environment
            step_result = env.step(action)
            
            # Handle different possible return types for step
            if isinstance(step_result, tuple):
                next_state = step_result[0]
                reward = step_result[1]
                terminated = step_result[2]
                truncated = step_result[3]
                # Optionally include info if it exists
                info = step_result[4] if len(step_result) > 4 else {}
            else:
                # Fallback if step returns something unexpected
                print("Unexpected step result type")
                break
            
            done = terminated or truncated
            
            # Store in replay buffer
            replay_buffer.add((state, action, reward, next_state, done))
            
            state = next_state
            episode_reward += reward
            
            # Update TD3 agent
            if replay_buffer.size >= 1000:  # Start updating after collecting some experiences
                agent.update(
                    replay_buffer, 
                    n_iter=1,  # Number of update iterations
                    batch_size=64, 
                    gamma=0.99,  # Discount factor
                    polyak=0.995,  # Polyak averaging coefficient
                    policy_noise=0.2,  # Noise added to actions during critic update
                    noise_clip=0.5,  # Limit for action noise
                    policy_delay=2  # Delayed policy update frequency
                )
            
            if done:
                break
        
        print(f"Episode {episode+1}/{num_episodes}, Total Reward: {episode_reward}")
    
    return agent

def main():
    # Import the necessary classes
    from rl_cardiac.tcn_model import TCN_config
    from rl_cardiac.cardiac_model import CardiacModel_Env

    # Set up the environment
    rat_type = 'healthy_stable'
    tcn_model = TCN_config(rat_type)
    env = CardiacModel_Env(tcn_model, rat_type)
    
    # Create log directory
    log_dir = "./logs/"
    os.makedirs(log_dir, exist_ok=True)
    env = Monitor(env, log_dir)

    # Train the TD3 agent
    trained_agent = train_td3(env)

    # Optional: Save the trained model
    os.makedirs('models', exist_ok=True)
    trained_agent.save('models', 'cardiac_td3')

if __name__ == '__main__':
    main()



script_dir /home/rl_course/rl_cardiac
[tcn_model/healthy_stable]
tcn_model healthy_stable
Load model from /home/rl_course/rl_cardiac/Healthy_500samples/TCN_1
[cardiac_model/healthy_stable]
state_dim:  4
action_dim: 6
max_action: 1.0
seed:  None
1/1 [==============================] - 0s 13ms/step
Episode 1/1000, Total Reward: 22.524314985498403
seed:  None
1/1 [==============================] - 0s 14ms/step
Episode 2/1000, Total Reward: 22.524314985498403
seed:  None
1/1 [==============================] - 0s 13ms/step
Episode 3/1000, Total Reward: 22.524314985498403
seed:  None
1/1 [==============================] - 0s 13ms/step
Episode 4/1000, Total Reward: 22.524314985498403
seed:  None
1/1 [==============================] - 0s 12ms/step
Episode 5/1000, Total Reward: 22.524314985498403
seed:  None
1/1 [==============================] - 0s 14ms/step
Episode 6/1000, Total Reward: 22.524314985498403
seed:  None
1/1 [==============================] - 0s 11ms/step
Episode 7/1000, Total Rew

In [ ]:
state_dim
action_dim
max_action

In [ ]:
###plottig helper
from stable_baselines3.common import results_plotter

# Helper from the library
results_plotter.plot_results(
    [log_dir], 1e5, results_plotter.X_TIMESTEPS, "TD3 cardiac_model"
)

In [ ]:
def moving_average(values, window):
    """
    Smooth values by doing a moving average
    :param values: (numpy array)
    :param window: (int)
    :return: (numpy array)
    """
    weights = np.repeat(1.0, window) / window
    return np.convolve(values, weights, "valid")


def plot_results(log_folder, title="Learning Curve"):
    """
    plot the results

    :param log_folder: (str) the save location of the results to plot
    :param title: (str) the title of the task to plot
    """
    x, y = ts2xy(load_results(log_folder), "timesteps")
    y = moving_average(y, window=50)
    # Truncate x
    x = x[len(x) - len(y) :]

    fig = plt.figure(title)
    plt.plot(x, y)
    plt.xlabel("Number of Timesteps")
    plt.ylabel("Rewards")
    plt.title(title + " Smoothed")
    plt.show()

In [ ]:
plot_results(log_dir)